In [2]:
from tokens import *

In [3]:
tokens = {
    "si":si,
    "sino":sino,
    "entonces":entonces,
    "finsi":finsi,
    "repetir":repetir,
    "hasta":hasta,
    "equal":equal,
    "leer":leer,
    "mostar":mostrar,
    "func":func,
    "finfunc":finfunc,
    "numero":numero,
    "(":parentesisApertura,
    ")":parentesisCerrado,
    ";":punto_coma,
    "oprel":oprel,
    "opsuma":opsuma,
    "opmult":opmult,
    "id":var
    }

In [4]:
def lexer(programa):
    # Eliminar espacio en blanco antes y despues del programa, y agregar un
    # espacio al final para poder agregar el ultimo token.
    programa = programa.strip()
    programa += " "

    tokens_out = []  # La salida del programa
    tokens_posibles = [t for t in tokens]  # los tokens posibles para el lexema
    # tokens posibles para un caracter mas
    tokens_posibles_1mas = tokens_posibles.copy()
    lexema = ""
    lexema1mas = ""
    for i in range(len(programa)):

        # Se incrementan el lexema y el lexema1mas en un caracter
        lexema = lexema1mas
        lexema1mas = lexema1mas + programa[i]

        # Si tokens_posibles es vacío esto es porque se acaba de agregar un
        # lexema a la salida, entonces hay que agregar los tokens posibles con
        # para el unico caracter que contiene el lexema
        if tokens_posibles == []:
            for token in tokens:
                if tokens[token].recognize_lexeme(lexema) != "Trampa":
                    tokens_posibles.append(token)
            tokens_posibles_1mas = tokens_posibles.copy()
        else:
            # Si tokens_posibles no esta vacio, solo se copian los tokens
            # posibles con un caracter mas del ciclo anterior
            tokens_posibles = tokens_posibles_1mas.copy()

        # si el lexema es solo un espacio en blanco o un salto de linea, saltear
        # un caracter y no hacer nada.
        if lexema == " " or lexema == "\n" or lexema == "\t":
            lexema1mas = programa[i]
            continue

        # contiene los tokens en estado final con el lexema (se llena en el ciclo)
        tokens_final = []
        for token in tokens_posibles:
            # si el afd para un token con el lexema1mas cae en estado trampa,
            # eliminar ese token de los tokens_posibles_1mas
            estado_1mas = tokens[token].recognize_lexeme(lexema1mas)
            if estado_1mas == "Trampa":
                tokens_posibles_1mas.remove(token)
            # si el afd para un token con el lexema cae en un estado final,
            # agregarlo a los tokens_finales
            estado_actual = tokens[token].recognize_lexeme(lexema)
            if estado_actual == "Final":
                tokens_final.append(token)

        # si no hay ningun token posible con un caracter mas, y hay al menos un
        # token en estado final, agregar ese token a la salida del lexer.
        if tokens_posibles_1mas == []:
            if tokens_final != []:
                tokens_out.append((tokens_final[0],lexema))
                lexema1mas = programa[i]
                tokens_posibles = []
            else:
                # En caso de que no haya ningun token posible con un caracter
                # mas pero tampoco haya ninguno en estado final, terminar el
                # lexer con un error de token invalido.
                raise ValueError("Token invalido")

    return tokens_out

In [5]:
lexer("i equal 2")

[('id', 'i'), ('equal', 'equal'), ('numero', '2')]

In [8]:
token_list=lexer("repetir x equal 5 finsi 123x x>=3 fin finfunc func")
print(token_list)

[('repetir', 'repetir'), ('id', 'x'), ('equal', 'equal'), ('numero', '5'), ('finsi', 'finsi'), ('numero', '123'), ('id', 'x'), ('id', 'x'), ('oprel', '>='), ('numero', '3'), ('id', 'fin'), ('finfunc', 'finfunc'), ('func', 'func')]


In [25]:

ElementoIzq=0
for N in range(len(token_list)):
    print(token_list[N][ElementoIzq])

repetir
id
equal
numero
finsi
numero
id
id
oprel
numero
id
finfunc
func


In [29]:
P = {
    'Program': [
        ['ListaSentencias']
    ],
    'ListaSentencias': [
        ['Sentencia','ListaSentecias_Prime']
    ],
    'ListaSentecias_Prime' :[
        [';', 'Sentencia'],['λ']
    ],
    'Sentencia': [
        ['SentenciaSi'],
        ['SentenciaRepetir'],
        ['SentenciaAsig'],
        ['SentenciaLeer'],
        ['SentenciaMostrar'],
        ['SentenciaFun']
    ],
    'SentenciaSi': [ 
        ['si', 'Expresion', 'entonces', 'ListaSentencias', 'SentenciaSi_Prime'] 
    ],
    'SentenciaSi_Prime' : [ 
        ['sino', 'ListaSentencias', 'finsi'], ['finsi'] 
    ],
    'SentenciaRepetir': [
        ['repetir', 'ListaSentencias', 'hasta', 'Expresion']
    ],
    'SentenciaAsig': [
        ['id', 'equal', 'Expresion']
    ],
    'SentenciaLeer': [
        ['leer', 'id']
    ],
    'SentenciaMostrar': [
        ['mostrar', 'Expresion']
    ],
    'SentenciaFun': [
        ['func', 'Proc', 'finfunc']
    ],
    'Proc': [
        ['id', '(', 'ListaPar', ')', 'ListaSentencias']
    ],
    'ListaPar': [
        ['id','ListaPar_Prime']
    ],
    'ListaPar_Prime':[
        [';', 'id','ListaPar_Prime'],['λ']
    ],
    'Expresion': [
        ['Expresion2', 'Expresion_Prime']
    ],
    'Expresion_Prime' : [
        ['oprel', 'Expresion2'],['λ']
    ],
    'Expresion2': [
        ['Termino','Expresion2_Prime']
    ],
    'Expresion2_Prime':[
        ['opsuma', 'Termino','Expresion2_Prime'],['λ']
    ],
    'Termino': [
        ['Factor','Termino_Prime']
    ],
    'Termino_Prime':[
        [ 'opmult', 'Factor','Termino_Prime'],['λ']     
    ],
    'Factor': [
        ['(', 'Expresion', ')'],
        ['num'],
        ['id']
    ]
}


In [37]:


for cuerpo in P['Sentencia']:
    print(cuerpo)
    for simbolo in cuerpo:
        print(simbolo)

['SentenciaSi']
SentenciaSi
['SentenciaRepetir']
SentenciaRepetir
['SentenciaAsig']
SentenciaAsig
['SentenciaLeer']
SentenciaLeer
['SentenciaMostrar']
SentenciaMostrar
['SentenciaFun']
SentenciaFun


In [42]:
[1,2] + [('EOF','EOF')]

[1, 2, ('EOF', 'EOF')]

In [45]:
SD = {
    'Programa': {'si':['ListasSentencias'],'no':['ListasSentencias'],'wow':['ListasSentencias']}
}

In [49]:
SD['Programa']['si']

['ListasSentencias']